# A simple tutorial pipleline for paramterization of M dwarf stars  

Jiadong LI (mailto:jdli@nao.cas.cn), Created on Fri. Mar 13 2020

In [1]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-
import os, sys, time
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
from astropy.io import fits
from astropy.table import Table
from slam import Slam
from slam.normalization import normalize_spectrum, normalize_spectra_block
from preprocess import LAMOST

In [2]:
# specify a wavelength range from LAMOST DR5 (R~1800) spectra as useful pixels

wave_interp = np.load("./models/wave_interp_R1800.npz")['wave']

# Preprocess: load all spectra data
obs_dir = "./data/"
obs_spec = LAMOST(root_dir=obs_dir, wave_resamp=wave_interp)


fluxes, invars = [], []
for i in tqdm(range(len(obs_spec))):
    fluxes += [obs_spec[i]['flux_resamp']]
    invars += [obs_spec[i]['invar_resamp']]
    
fluxes, invars = np.array(fluxes), np.array(invars)

100%|██████████| 3/3 [00:00<00:00, 39.12it/s]


In [3]:
# normalization of each spetra

fluxes_norm, fluxes_cont = normalize_spectra_block(wave_interp, fluxes, 
                                                   (6147., 8910.), 10., p=(1E-8, 1E-7), q=0.7, 
                                                   eps=1E-19, rsv_frac=2., n_jobs=2, verbose=5)

invars_norm = fluxes_cont**2*invars

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   3 out of   3 | elapsed:    1.6s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   3 out of   3 | elapsed:    1.6s finished


In [4]:
# load SLAM model trained by BT-Settl synthetic spectra

k_bt = Slam.load_dump("./models/btsettl.dump")

In [ ]:
label_init = k_bt.predict_labels_quick(fluxes_norm, invars_norm, n_jobs=2)

results_pred = k_bt.predict_labels_multi(label_init, fluxes_norm, invars_norm)
label_pred = np.array([label['x'] for label in results_pred])

TEFF_BT = label_pred[:,0]

@Slam.heal_the_world: IVAR of 0 spectra need to be reset


@Slam.heal_the_world: FLUX of 0 spectra need to be reset

@Slam.heal_the_world: IVAR of 0 spectra need to be reset


@Slam.heal_the_world: FLUX of 0 spectra need to be reset

@SLAM: NMSE is not available and will be calculated now!
